In [2]:
import pprint
import sys
import pickle
import pandas as pd
import numpy as np
from time import time
from tester import dump_classifier_and_data

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectKBest, chi2

import pylab as pl
from feature_format import featureFormat
from feature_format import targetFeatureSplit

import matplotlib.pyplot as plt

/Users/Haleh/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
# %load poi_id.py
#!/usr/bin/python
sys.path.append("../tools/")

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

In [4]:
pprint.pprint (data_dict.keys())
print ("There are ", len(data_dict.keys()), " executives in Enron Dataset.")


['METTS MARK',
 'BAXTER JOHN C',
 'ELLIOTT STEVEN',
 'CORDES WILLIAM R',
 'HANNON KEVIN P',
 'MORDAUNT KRISTINA M',
 'MEYER ROCKFORD G',
 'MCMAHON JEFFREY',
 'HORTON STANLEY C',
 'PIPER GREGORY F',
 'HUMPHREY GENE E',
 'UMANOFF ADAM S',
 'BLACHMAN JEREMY M',
 'SUNDE MARTIN',
 'GIBBS DANA R',
 'LOWRY CHARLES P',
 'COLWELL WESLEY',
 'MULLER MARK S',
 'JACKSON CHARLENE R',
 'WESTFAHL RICHARD K',
 'WALTERS GARETH W',
 'WALLS JR ROBERT H',
 'KITCHEN LOUISE',
 'CHAN RONNIE',
 'BELFER ROBERT',
 'SHANKMAN JEFFREY A',
 'WODRASKA JOHN',
 'BERGSIEKER RICHARD P',
 'URQUHART JOHN A',
 'BIBI PHILIPPE A',
 'RIEKER PAULA H',
 'WHALEY DAVID A',
 'BECK SALLY W',
 'HAUG DAVID L',
 'ECHOLS JOHN B',
 'MENDELSOHN JOHN',
 'HICKERSON GARY J',
 'CLINE KENNETH W',
 'LEWIS RICHARD',
 'HAYES ROBERT E',
 'MCCARTY DANNY J',
 'KOPPER MICHAEL J',
 'LEFF DANIEL P',
 'LAVORATO JOHN J',
 'BERBERIAN DAVID',
 'DETMERING TIMOTHY J',
 'WAKEHAM JOHN',
 'POWERS WILLIAM',
 'GOLD JOSEPH',
 'BANNANTINE JAMES M',
 'DUNCAN JOHN H'

## Task 1: Select what features you'll use.
features_list is a list of strings, each of which is a feature name.<br>
The first feature must be "poi".<br>
You will need to use more features

In [5]:
### Task 1: Select what features you'll use.

features_list = ['poi',
                 'salary',
                 'salary_to_avg',
                 'deferral_payments',
                 'total_payments',
                 'loan_advances',
                 'bonus',
                 'restricted_stock_deferred',
                 'exercised_stock_options',
                 'from_poi_to_this_person',
                 'from_this_person_to_poi']


## Task 2: Remove outliers
based on mini-project in outlier lessons, i know there is a key "Total", which holds the total value for all other values. <br>
Another one is the key "THE TRAVEL AGENCY IN THE PARK".<br>
I'm going to exclude both of them



In [6]:
pprint.pprint (data_dict["TOTAL"], width=1)
pprint.pprint (data_dict["THE TRAVEL AGENCY IN THE PARK"])

{'bonus': 97343619,
 'deferral_payments': 32083396,
 'deferred_income': -27992891,
 'director_fees': 1398517,
 'email_address': 'NaN',
 'exercised_stock_options': 311764000,
 'expenses': 5235198,
 'from_messages': 'NaN',
 'from_poi_to_this_person': 'NaN',
 'from_this_person_to_poi': 'NaN',
 'loan_advances': 83925000,
 'long_term_incentive': 48521928,
 'other': 42667589,
 'poi': False,
 'restricted_stock': 130322299,
 'restricted_stock_deferred': -7576788,
 'salary': 26704229,
 'shared_receipt_with_poi': 'NaN',
 'to_messages': 'NaN',
 'total_payments': 309886585,
 'total_stock_value': 434509511}
{'bonus': 'NaN',
 'deferral_payments': 'NaN',
 'deferred_income': 'NaN',
 'director_fees': 'NaN',
 'email_address': 'NaN',
 'exercised_stock_options': 'NaN',
 'expenses': 'NaN',
 'from_messages': 'NaN',
 'from_poi_to_this_person': 'NaN',
 'from_this_person_to_poi': 'NaN',
 'loan_advances': 'NaN',
 'long_term_incentive': 'NaN',
 'other': 362096,
 'poi': False,
 'restricted_stock': 'NaN',
 'restri

In [7]:
data_dict.pop("TOTAL")
data_dict.pop("THE TRAVEL AGENCY IN THE PARK")

{'bonus': 'NaN',
 'deferral_payments': 'NaN',
 'deferred_income': 'NaN',
 'director_fees': 'NaN',
 'email_address': 'NaN',
 'exercised_stock_options': 'NaN',
 'expenses': 'NaN',
 'from_messages': 'NaN',
 'from_poi_to_this_person': 'NaN',
 'from_this_person_to_poi': 'NaN',
 'loan_advances': 'NaN',
 'long_term_incentive': 'NaN',
 'other': 362096,
 'poi': False,
 'restricted_stock': 'NaN',
 'restricted_stock_deferred': 'NaN',
 'salary': 'NaN',
 'shared_receipt_with_poi': 'NaN',
 'to_messages': 'NaN',
 'total_payments': 362096,
 'total_stock_value': 'NaN'}

# Task 3: Create new feature(s)
Store to my_dataset for easy export below.

In [8]:
total = 0.0
avg=0.0
for k,v in data_dict.iteritems():
    if v['salary'] == "NaN" or v['salary']<0:
        v['salary_to_avg'] = "NaN"
    else:
        total += v['salary']
avg= total/94        
print avg

284087.542553


In [9]:
for key, value in data_dict.iteritems():
    if value["salary"] == "NaN" :
        value['salary_to_avg'] = "NaN"
    else:
        value['salary_to_avg'] = float(value['salary']) / float(avg)

In [10]:
# for k,v in data_dict.iteritems():
#     print v['salary_to_avg']

In [11]:
my_dataset = data_dict
### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

In [12]:
df = pd.DataFrame.from_dict(my_dataset, orient = 'index')
df = df[features_list]
df = df.replace('NaN', np.nan)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 144 entries, ALLEN PHILLIP K to YEAP SOON
Data columns (total 11 columns):
poi                          144 non-null bool
salary                       94 non-null float64
salary_to_avg                94 non-null float64
deferral_payments            38 non-null float64
total_payments               123 non-null float64
loan_advances                3 non-null float64
bonus                        81 non-null float64
restricted_stock_deferred    17 non-null float64
exercised_stock_options      101 non-null float64
from_poi_to_this_person      86 non-null float64
from_this_person_to_poi      86 non-null float64
dtypes: bool(1), float64(10)
memory usage: 12.5+ KB


# Task 4: Try a varity of classifiers
Please name your classifier clf for easy export below.
Note that if you want to do PCA or other multi-stage operations,
you'll need to use Pipelines. For more info:
http://scikit-learn.org/stable/modules/pipeline.html

In [13]:
# Provided to give you a starting point. Try a variety of classifiers.

data = featureFormat(my_dataset, features_list)
labels, features = targetFeatureSplit(data)
features_train, features_test, labels_train, labels_test = train_test_split(features, labels,  
                                                                            test_size=0.3, random_state=42)

# Data Prep

## Feature Scaling 

In [14]:
# # minmax scaler
scaler = MinMaxScaler()
scaled_feature_train = scaler.fit_transform(features_train)
scaled_feature_test = scaler.fit_transform(features_test)

# Algorithms

In [15]:
clf_knc = KNeighborsClassifier()
model_knc = clf_knc.fit(scaled_feature_train, labels_train) 
pred_knc = model_knc.predict(scaled_feature_test)
acc_knc= accuracy_score(pred_knc, labels_test)
print acc_knc


0.837209302326


In [16]:
clf_dt = DecisionTreeClassifier()
clf_dt = clf_dt.fit(scaled_feature_train, labels_train) 
pred_dt = clf_dt.predict(scaled_feature_test)
acc_dt= accuracy_score(pred_dt, labels_test)

print acc_dt

0.813953488372


In [17]:
#Creat the pipeline
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=100, random_state = 42)

pipe = Pipeline([
    ('reduce_dim', SelectKBest()),
    ('classify', DecisionTreeClassifier(splitter='best', random_state=42))
])

param_grid_DT = [
    {
        'reduce_dim': [SelectKBest()],
        'reduce_dim__k': [3,6],
        'classify__criterion':['gini', 'entropy'],
        'classify__class_weight':[None, 'balanced'],
        'classify__min_samples_split' : [2,3,4,5],
        'classify__max_leaf_nodes': [None, 5],
        'classify__min_samples_leaf': [4, 2]
    },
]

grid = GridSearchCV(pipe, cv=sss, param_grid=param_grid_DT, scoring='f1')
gs = grid.fit(scaled_feature_train, labels_train)
clf =gs.best_estimator_




/Users/Haleh/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/Haleh/anaconda/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [4] are constant.
  UserWarning)
/Users/Haleh/anaconda/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [18]:
dump_classifier_and_data(clf, my_dataset, features_list)
from tester import dump_classifier_and_data, test_classifier
test_classifier(clf, my_dataset, features_list)

Pipeline(steps=[('reduce_dim', SelectKBest(k=3, score_func=<function f_classif at 0x11ae226e0>)), ('classify', DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=5, min_impurity_split=1e-07,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'))])
	Accuracy: 0.84573	Precision: 0.29872	Recall: 0.11650	F1: 0.16763	F2: 0.13269
	Total predictions: 15000	True positives:  233	False positives:  547	False negatives: 1767	True negatives: 12453



In [19]:
#Creat the pipeline
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier

sss = StratifiedShuffleSplit(n_splits=100, random_state = 42)
pipe_KNC = Pipeline([
    ('reduce_dim', SelectKBest()),
    ('classify', KNeighborsClassifier())
])
param_grid_KNC = [
    {
        'reduce_dim': [SelectKBest()],
        'reduce_dim__k': [3, 6],
        'classify__algorithm' : ['auto'], 
        'classify__leaf_size' : [30, 50],
        'classify__n_neighbors' : [5, 10], 
        'classify__p' : [1, 2], 
        'classify__weights' : ['uniform', 'distance'],
        'classify__algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute']
    },
]

#gridsearch
grid_KNC = GridSearchCV(pipe_KNC, cv=sss, param_grid=param_grid_KNC, scoring='recall')
gs_KNC = grid_KNC.fit(scaled_feature_train, labels_train)
clf_KNC =gs_KNC.best_estimator_


In [20]:
dump_classifier_and_data(clf_KNC, my_dataset, features_list)
from tester import dump_classifier_and_data, test_classifier
test_classifier(clf_KNC, my_dataset, features_list)

Pipeline(steps=[('reduce_dim', SelectKBest(k=6, score_func=<function f_classif at 0x11ae226e0>)), ('classify', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=1,
           weights='distance'))])
	Accuracy: 0.90087	Precision: 0.75779	Recall: 0.37700	F1: 0.50351	F2: 0.41912
	Total predictions: 15000	True positives:  754	False positives:  241	False negatives: 1246	True negatives: 12759

